# Classifying Invasive and Endangered Species

## Imports & Libraries

In [155]:
import numpy as np
import pandas as pd

# Preprocessing
from sklearn.preprocessing import OneHotEncoder

# Modeling
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from matplotlib import pyplot as plt

In [156]:
mobil = pd.read_excel('mobil.xlsx')

### Trends
- temp by time

In [157]:
# Using plotly.express
import plotly.express as px

df = mobil
fig = px.line(df, x='Date', y="Water temperature (ºC)")
fig.show()

In [158]:
# Counting observations
mobil.groupby(mobil['Date'].dt.year)['Water temperature (ºC)'].count()

Date
2011.0     44
2012.0    440
2013.0    242
2014.0    245
2015.0    264
2016.0    231
2017.0    264
2018.0    108
2019.0     91
2020.0     73
Name: Water temperature (ºC), dtype: int64

In [159]:
# When the temperature varies from the top/mode/norm is it above or below?
mobil.groupby(mobil['Date'].dt.month)['Water temperature (ºC)'].describe()

,count,unique,top,freq
Date,,,,
1.0,209.0,9.0,15.0,93.0
2.0,181.0,10.0,14.0,38.0
3.0,143.0,6.0,15.0,44.0
4.0,174.0,7.0,15.0,55.0
5.0,188.0,9.0,17.0,43.0
6.0,165.0,7.0,19.0,44.0
7.0,192.0,4.0,19.0,77.0
8.0,152.0,7.0,17.5,33.0
9.0,126.0,7.0,18.0,27.0


## Pre-Processing
- Continuous: 'Water temperature (ºC)', 
- 

In [160]:
# Continuous data
mobil.iloc[:,:9].dtypes

Date                            datetime64[ns]
Hour                                    object
Tide                                   float64
Weather Condition                       object
Water temperature (ºC)                  object
Sampler                                 object
Zone                                    object
Supratidal/Middle Intertidal            object
Substrate                               object
dtype: object

In [161]:
mobil.head(2)

,Date,Hour,Tide,Weather Condition,Water temperature (ºC),Sampler,Zone,Supratidal/Middle Intertidal,Substrate,Chthamalus sp.,...,Alga castanha carnuda,Alga castanha tufosa,Alga verde/azul - lavanda,Sphacelaria rigidula (pompons castanhos),Cystoseira sp.,Laminaria sp.,TOTAL2,observações,Coluna1,Coluna2
0,2011-11-28,12:20:00,0.6,Clear sky,16,AF SF,D,Supra,Rock,7,...,0.0,0.0,0.0,0.0,0,0.0,93.3,0,0,0.0
1,2011-11-28,12:35:00,0.6,Clear sky,16,AF SF,D,Supra,Rock,0,...,0.0,0.0,0.0,0.0,0,0.0,0.1,,0,0.0


In [162]:
mobil['Hour'][0]

datetime.time(12, 20)

In [163]:
mobil_features = pd.get_dummies(mobil[['Weather Condition', 'Zone', 'Supratidal/Middle Intertidal', 'Substrate']])


In [164]:
mobil_features['Tide'] = mobil['Tide'].astype(float)
mobil_features.head(3)

,Weather Condition_Clear sky,Weather Condition_Cloudy,Weather Condition_Fairly Cloudy,Weather Condition_Rain,Weather Condition_Sunny,Weather Condition_Sunny and Windy,Zone_A,Zone_B,Zone_D,Zone_E,...,Substrate_Rock/Sand/Pebble,Substrate_Rock/Sand/Sea,Substrate_Rock/Sea,Substrate_Sand,Substrate_Sand/ Pebble/Canal,Substrate_Sand/Canal,Substrate_Sand/Pebble,Substrate_Sand/Sea,Substrate_Sea,Tide
0,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.6
1,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.6
2,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.6


## Predicting Abundance

### Regression Models
- Linear regression/ multivariate linear regression
- Polynomial regression
- Logistic regression '

write up on feature importance

In [165]:
mobil.columns

Index(['Date', 'Hour', 'Tide', 'Weather Condition', 'Water temperature (ºC)',
       'Sampler', 'Zone', 'Supratidal/Middle Intertidal', 'Substrate',
       'Chthamalus sp.',
       ...
       'Alga castanha carnuda', 'Alga castanha tufosa',
       'Alga verde/azul - lavanda', 'Sphacelaria rigidula (pompons castanhos)',
       'Cystoseira sp.', 'Laminaria sp.', 'TOTAL2', 'observações', 'Coluna1',
       'Coluna2'],
      dtype='object', length=105)

In [167]:
# train and test sets

X = mobil_features
y = mobil['Abundance (ind/m2)'] #unable to find column?

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

KeyError: 'Abundance'